In [1]:
import pandas as pd
import numpy as np
import nltk
nltk.download('wordnet')
import re
from bs4 import BeautifulSoup

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/brinkley97/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## Read Data

In [2]:
dataset = "../datasets/amazon_reviews_us_Office_Products_v1_00.csv"
amazon_reviews_df = pd.read_csv(dataset, on_bad_lines='skip', low_memory=False)

In [3]:
amazon_reviews_df

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
US,43081963,R18RVCKGH1SSI9,B001BM2MAC,307809868,Scotch Cushion Wrap 7961,12 Inches x 100 Feet,Office Products,5,0,0,N,Y,Five Stars,Great product.,2015-08-31
US,21143145,R2J8AWXWTDX2TF,B00RTMUHDW,529689027,Amram Tagger Standard Tag Attaching Tagging Gu...,Office Products,5,0,0,N,Y,but I am sure I will like it.,Haven't used yet,but I am sure I will like it.,2015-08-31
US,9109358,R2YWMQT2V11XYZ,B00MOPAG8K,863351797,KITLEX40X2592UNV21200 - Value Kit - Lexmark 40...,Office Products,5,0,0,N,N,Five Stars,Gold plated fusers are the best! It will never...,2015-08-31,NaN
US,9967215,R1V2HYL6OI9V39,B003AHIK7U,383470576,Bible Dry Highlighting Kit (Set of 4),Office Products,5,6,6,N,Y,Won't bleed through pages!,I have used these highlighters for my bible fo...,2015-08-31,NaN
US,11234247,R3BLQBKUNXGFS4,B006TKH2RO,999128878,Parker Ingenuity Large Black Rubber & Metal CT...,Office Products,5,0,0,N,Y,Nice pen!,Heavy pen that writes very well. I've had it ...,2015-08-31,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
US,50937664,R3SLCLDA2C622E,B00000J4FS,42180401,Palm V Hand held PDA,Office Products,5,0,0,N,N,If you can use all of its features,its perfect.,This palm is by no means for a hobbiest; its a...,1999-06-15
US,53048056,R1UV095FUPI31M,B00000J4FS,42180401,Palm V Hand held PDA,Office Products,5,1,1,N,N,I love mine!,I had one of the earlier Palm Organizers. The...,1999-06-05,NaN
US,52937711,R3R9EE77PAPM9M,B00000J4FS,42180401,Palm V Hand held PDA,Office Products,4,0,0,N,N,Most handy PDA available,In addition to being easy to use (like all the...,1999-06-03,NaN
US,52188548,R1F3SRK9MHE6A3,B00000DM9M,223408988,PalmOne III Leather Belt Clip Case,Office Products,4,18,18,N,N,Attaches the Palm Pilot like an appendage,Although the Palm Pilot is thin and compact it...,enter and review data at any time. This espe...,1998-11-30


In [4]:
amazon_reviews_copy_df = amazon_reviews_df.copy()

In [5]:
amazon_reviews_copy_df.reset_index(inplace=True)

In [6]:
amazon_reviews_copy_df

,index,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,43081963,R18RVCKGH1SSI9,B001BM2MAC,307809868,Scotch Cushion Wrap 7961,12 Inches x 100 Feet,Office Products,5,0,0,N,Y,Five Stars,Great product.,2015-08-31
1,US,21143145,R2J8AWXWTDX2TF,B00RTMUHDW,529689027,Amram Tagger Standard Tag Attaching Tagging Gu...,Office Products,5,0,0,N,Y,but I am sure I will like it.,Haven't used yet,but I am sure I will like it.,2015-08-31
2,US,9109358,R2YWMQT2V11XYZ,B00MOPAG8K,863351797,KITLEX40X2592UNV21200 - Value Kit - Lexmark 40...,Office Products,5,0,0,N,N,Five Stars,Gold plated fusers are the best! It will never...,2015-08-31,NaN
3,US,9967215,R1V2HYL6OI9V39,B003AHIK7U,383470576,Bible Dry Highlighting Kit (Set of 4),Office Products,5,6,6,N,Y,Won't bleed through pages!,I have used these highlighters for my bible fo...,2015-08-31,NaN
4,US,11234247,R3BLQBKUNXGFS4,B006TKH2RO,999128878,Parker Ingenuity Large Black Rubber & Metal CT...,Office Products,5,0,0,N,Y,Nice pen!,Heavy pen that writes very well. I've had it ...,2015-08-31,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1329229,US,50937664,R3SLCLDA2C622E,B00000J4FS,42180401,Palm V Hand held PDA,Office Products,5,0,0,N,N,If you can use all of its features,its perfect.,This palm is by no means for a hobbiest; its a...,1999-06-15
1329230,US,53048056,R1UV095FUPI31M,B00000J4FS,42180401,Palm V Hand held PDA,Office Products,5,1,1,N,N,I love mine!,I had one of the earlier Palm Organizers. The...,1999-06-05,NaN
1329231,US,52937711,R3R9EE77PAPM9M,B00000J4FS,42180401,Palm V Hand held PDA,Office Products,4,0,0,N,N,Most handy PDA available,In addition to being easy to use (like all the...,1999-06-03,NaN
1329232,US,52188548,R1F3SRK9MHE6A3,B00000DM9M,223408988,PalmOne III Leather Belt Clip Case,Office Products,4,18,18,N,N,Attaches the Palm Pilot like an appendage,Although the Palm Pilot is thin and compact it...,enter and review data at any time. This espe...,1998-11-30


## Keep Reviews and Ratings

In [7]:
reviews_ratings_df = amazon_reviews_copy_df.loc[0:, ['star_rating', 'review_headline', 'review_body']]
reviews_ratings_df

,star_rating,review_headline,review_body
0,5,Five Stars,Great product.
1,0,Haven't used yet,but I am sure I will like it.
2,0,Gold plated fusers are the best! It will never...,2015-08-31
3,6,I have used these highlighters for my bible fo...,2015-08-31
4,0,Heavy pen that writes very well. I've had it ...,2015-08-31
...,...,...,...
1329229,0,its perfect.,This palm is by no means for a hobbiest; its a...
1329230,1,I had one of the earlier Palm Organizers. The...,1999-06-05
1329231,0,In addition to being easy to use (like all the...,1999-06-03
1329232,18,Although the Palm Pilot is thin and compact it...,enter and review data at any time. This espe...


 ## We form three classes and select 20000 reviews randomly from each class.



In [8]:
def separate_reviews_by_rating(df: pd.DataFrame, rating_col: str, threshold: int, sentiment_type: str):

    df[rating_col] = pd.to_numeric(df[rating_col], errors='coerce')


    if sentiment_type == 'positive_sentiment':
        positive_review_threshold = df[rating_col] > threshold
        df = df[positive_review_threshold]
        df[sentiment_type] = 1

    elif sentiment_type == 'negative_sentiment':
        positive_review_threshold = df[rating_col] <= threshold
        df = df[positive_review_threshold]
        df[sentiment_type] = 0

    elif sentiment_type == 'neutral_sentiment':
        positive_review_threshold = df[rating_col] == threshold
        df = df[positive_review_threshold]
        df[sentiment_type] = 3
        
    return df

In [9]:
pos_df = separate_reviews_by_rating(reviews_ratings_df, 'star_rating', 3, 'positive_sentiment')

/var/folders/fz/zn5r8vq12nv5p23dtlr15sk40000gn/T/ipykernel_33730/2403254103.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[sentiment_type] = 1


In [10]:
pos_df

,star_rating,review_headline,review_body,positive_sentiment
0,5.0,Five Stars,Great product.,1
3,6.0,I have used these highlighters for my bible fo...,2015-08-31,1
16,5.0,Five Stars,Perfect for what I need it for! !!!,1
19,5.0,Five Stars,Depend on them.,1
22,5.0,Five Stars,Perfect,1
...,...,...,...,...
1329208,6.0,I use the HP Jornada 680 to keep all of my bus...,1999-07-14,1
1329209,8.0,I loved the digital AM/FM radio...it is much m...,1999-07-14,1
1329220,6.0,You can't find a better 36 bit scanner for tha...,this is the one.,1
1329221,14.0,This simple leather case keeps my Palm V safe ...,1999-07-01,1


In [11]:
neg_df = separate_reviews_by_rating(reviews_ratings_df, 'star_rating', 2, 'negative_sentiment')

/var/folders/fz/zn5r8vq12nv5p23dtlr15sk40000gn/T/ipykernel_33730/2403254103.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[sentiment_type] = 0


In [12]:
neg_df

,star_rating,review_headline,review_body,negative_sentiment
1,0.0,Haven't used yet,but I am sure I will like it.,0
2,0.0,Gold plated fusers are the best! It will never...,2015-08-31,0
4,0.0,Heavy pen that writes very well. I've had it ...,2015-08-31,0
5,0.0,Not sure if they work but sent quickly and fit...,2015-08-31,0
6,0.0,Love these highlighters. They took a while to ...,but nonetheless I use them in every class I h...,0
...,...,...,...,...
1329228,0.0,This is probably the best portable computing d...,and I find it to be a huge pity that it canno...,0
1329229,0.0,its perfect.,This palm is by no means for a hobbiest; its a...,0
1329230,1.0,I had one of the earlier Palm Organizers. The...,1999-06-05,0
1329231,0.0,In addition to being easy to use (like all the...,1999-06-03,0


In [13]:
neu_df = separate_reviews_by_rating(reviews_ratings_df, 'star_rating', 3, 'neutral_sentiment')

/var/folders/fz/zn5r8vq12nv5p23dtlr15sk40000gn/T/ipykernel_33730/2403254103.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[sentiment_type] = 3


In [14]:
neu_df

,star_rating,review_headline,review_body,neutral_sentiment
147,3.0,OK but picture is not accurate.,Only complaint is the graphic is different tha...,3
171,3.0,Three Stars,Staples were fine. Good quality. Just too wide...,3
235,3.0,Cheap,vinyl-like feel. Not worth a quarter of the p...,3
335,3.0,Exactly what I needed in my classroom! I put t...,2015-08-31,3
557,3.0,Ok,It's a battery. It worked.,3
...,...,...,...,...
1329175,3.0,I really love the versions of Internet Explore...,and I'm a more comfortable mobile communicator.,3
1329194,3.0,I've been using an HP InkJet for several years...,and the output quality is truely unbelievable...,3
1329204,3.0,I came from a HP540 and it was considerably fa...,1999-07-15,3
1329212,3.0,This product is awesome! Print quality is gre...,1999-07-13,3


In [15]:
pos_rand_sampled_df = pos_df.sample(20000)
pos_rand_sampled_df

,star_rating,review_headline,review_body,positive_sentiment
1075394,5.0,SammyStuff,Love this shredder. It is durable and can han...,1
488457,5.0,Very good I like the black pen,Very good I like the black pen,1
633349,5.0,Very pleased with my purchase! The colors are ...,2014-10-04,1
579734,4.0,Four Stars,Good small tool case for the &#34;glove compar...,1
1060819,5.0,perfect,had problem finding this for my son at stores....,1
...,...,...,...,...
81760,5.0,Five Stars,Good price and works great!,1
1276734,4.0,Good deal on this binder it is plain but you d...,2009-08-30,1
397243,5.0,Five Stars,all some phones,1
995115,12.0,I received 4 discs with no instructions. I had...,in an attempt to get started. It looks like t...,1


In [16]:
neg_rand_sampled_df = neg_df.sample(20000)
neg_rand_sampled_df

,star_rating,review_headline,review_body,negative_sentiment
194810,0.0,it really works well,Iam hard of gearing and this is a godsend,0
635895,0.0,Great price for product and all that you get. ...,2014-10-02,0
89391,2.0,Two Stars,The holes not &#34;universal&#34; to all loose...,0
92167,0.0,Picture doesn't give accurate appearance. What...,http://www.amazon.com/gp/product/B001LDDLE2?r...,0
737018,0.0,Got one for me and my daughter. I use mine all...,2014-07-28,0
...,...,...,...,...
360704,0.0,I get the gron part namber i need you sand me ...,2015-02-26,0
822917,0.0,tremendous static not present on phone replace...,2014-04-17,0
327341,0.0,Cute and a good size for holding a decent amou...,makeup brushes and such.,0
673490,1.0,Loving this product and the fact that I can us...,2014-09-09,0


In [17]:
# reviews_ratings_df = pd.concat([pos_rand_sampled_df, neg_rand_sampled_df])

In [18]:
reviews_ratings_with_sentiment_df = pd.concat([pos_rand_sampled_df, neg_rand_sampled_df])

In [19]:
reviews_ratings_with_sentiment_df

,star_rating,review_headline,review_body,positive_sentiment,negative_sentiment
1075394,5.0,SammyStuff,Love this shredder. It is durable and can han...,1.0,NaN
488457,5.0,Very good I like the black pen,Very good I like the black pen,1.0,NaN
633349,5.0,Very pleased with my purchase! The colors are ...,2014-10-04,1.0,NaN
579734,4.0,Four Stars,Good small tool case for the &#34;glove compar...,1.0,NaN
1060819,5.0,perfect,had problem finding this for my son at stores....,1.0,NaN
...,...,...,...,...,...
360704,0.0,I get the gron part namber i need you sand me ...,2015-02-26,NaN,0.0
822917,0.0,tremendous static not present on phone replace...,2014-04-17,NaN,0.0
327341,0.0,Cute and a good size for holding a decent amou...,makeup brushes and such.,NaN,0.0
673490,1.0,Loving this product and the fact that I can us...,2014-09-09,NaN,0.0


# Data Cleaning

## Lower case
- NOTE: Not all reviews are a string. To solve,
    - [ ] Filter out non-strings when/before randomly sampling

In [20]:
def convert_reviews_to_lower_case(df: pd.DataFrame, col_name: str):
    """Convert all reviews to lower case

    Parameters
    ----------
    df: `pd.DataFrame`
        The data
    
    col_name: `str`
        Column with reviews

    Return
    ------
    df: `pd.DataFrame`
        An updated DataFrame with the lower cased reviews
    """
    
    lower_case_reviews = []
    updated_df = df.copy()
    text_reviews = df[col_name].values
    
    for text_reviews_idx in range(len(text_reviews)):
        text_review = text_reviews[text_reviews_idx]
        # print(text_reviews_idx, type(text_review), text_review)

        if type(text_review) != str:
            converted_str = str(text_review)
            # update_text_review = converted_str.lower()
            lower_case_reviews.append(update_text_review)
            # print(text_reviews_idx, update_text_review)
            # print()
        else:
            update_text_review = text_review.lower()
            lower_case_reviews.append(update_text_review)
            # print(text_reviews_idx, update_text_review)
            # print()

    update_col_name = col_name + '_lower_cased'
    updated_df[update_col_name] = lower_case_reviews
    
    # updated_df = df
    return updated_df

In [21]:
# reviews_ratings_with_sentiment_df

In [22]:
reviews_lc_hl_df = convert_reviews_to_lower_case(reviews_ratings_with_sentiment_df, 'review_headline')

In [23]:
# reviews_lc_hl_df.head(10)

In [24]:
reviews_lc_rb_df = convert_reviews_to_lower_case(reviews_lc_hl_df, 'review_body')

In [25]:
reviews_lc_rb_df

,star_rating,review_headline,review_body,positive_sentiment,negative_sentiment,review_headline_lower_cased,review_body_lower_cased
1075394,5.0,SammyStuff,Love this shredder. It is durable and can han...,1.0,NaN,sammystuff,love this shredder. it is durable and can han...
488457,5.0,Very good I like the black pen,Very good I like the black pen,1.0,NaN,very good i like the black pen,very good i like the black pen
633349,5.0,Very pleased with my purchase! The colors are ...,2014-10-04,1.0,NaN,very pleased with my purchase! the colors are ...,2014-10-04
579734,4.0,Four Stars,Good small tool case for the &#34;glove compar...,1.0,NaN,four stars,good small tool case for the &#34;glove compar...
1060819,5.0,perfect,had problem finding this for my son at stores....,1.0,NaN,perfect,had problem finding this for my son at stores....
...,...,...,...,...,...,...,...
360704,0.0,I get the gron part namber i need you sand me ...,2015-02-26,NaN,0.0,i get the gron part namber i need you sand me ...,2015-02-26
822917,0.0,tremendous static not present on phone replace...,2014-04-17,NaN,0.0,tremendous static not present on phone replace...,2014-04-17
327341,0.0,Cute and a good size for holding a decent amou...,makeup brushes and such.,NaN,0.0,cute and a good size for holding a decent amou...,makeup brushes and such.
673490,1.0,Loving this product and the fact that I can us...,2014-09-09,NaN,0.0,loving this product and the fact that i can us...,2014-09-09


## Remove HTML and URLs
- [ ] Verify by finding a specific entry with HTML, URL.

In [26]:
def remove_html_and_urls(df:pd.DataFrame, col_name: str):
    """Remove HTML and URLs from all reviews

    Parameters
    ----------
    df: `pd.DataFrame`
        The data
    
    col_name: `str`
        Column with reviews

    Return
    ------
    df: `pd.DataFrame`
        An updated DataFrame with the html_and_urls removed
    """
    
    url_pattern = re.compile(r'https?://\S+|www\. \S+')

    updated_reviews = []
    updated_df = df.copy()
    text_reviews = df[col_name].values

    for text_reviews_idx in range(len(text_reviews)):
        text_review = text_reviews[text_reviews_idx]

        if type(text_review) != str:
            updated_reviews.append(text_review)
        else:
            update_text_review = url_pattern.sub(r'', text_review)
            updated_reviews.append(update_text_review)

    updated_df[col_name] = updated_reviews
    return updated_df

In [27]:
reviews_no_html_urls_df = remove_html_and_urls(reviews_lc_rb_df, 'review_headline')

In [28]:
# reviews_no_html_urls_df

In [29]:
reviews_no_html_urls_rb_df = remove_html_and_urls(reviews_no_html_urls_df, 'review_body')

In [30]:
reviews_no_html_urls_rb_df

,star_rating,review_headline,review_body,positive_sentiment,negative_sentiment,review_headline_lower_cased,review_body_lower_cased
1075394,5.0,SammyStuff,Love this shredder. It is durable and can han...,1.0,NaN,sammystuff,love this shredder. it is durable and can han...
488457,5.0,Very good I like the black pen,Very good I like the black pen,1.0,NaN,very good i like the black pen,very good i like the black pen
633349,5.0,Very pleased with my purchase! The colors are ...,2014-10-04,1.0,NaN,very pleased with my purchase! the colors are ...,2014-10-04
579734,4.0,Four Stars,Good small tool case for the &#34;glove compar...,1.0,NaN,four stars,good small tool case for the &#34;glove compar...
1060819,5.0,perfect,had problem finding this for my son at stores....,1.0,NaN,perfect,had problem finding this for my son at stores....
...,...,...,...,...,...,...,...
360704,0.0,I get the gron part namber i need you sand me ...,2015-02-26,NaN,0.0,i get the gron part namber i need you sand me ...,2015-02-26
822917,0.0,tremendous static not present on phone replace...,2014-04-17,NaN,0.0,tremendous static not present on phone replace...,2014-04-17
327341,0.0,Cute and a good size for holding a decent amou...,makeup brushes and such.,NaN,0.0,cute and a good size for holding a decent amou...,makeup brushes and such.
673490,1.0,Loving this product and the fact that I can us...,2014-09-09,NaN,0.0,loving this product and the fact that i can us...,2014-09-09


## Remove Non-alphabetical characters
- [ ] If entry has no letters, leave blank?

In [31]:
def remove_non_alphabetical_characters(df:pd.DataFrame, col_name: str):
    """Remove Non-alphabetical characters from all reviews

    Parameters
    ----------
    df: `pd.DataFrame`
        The data
    
    col_name: `str`
        Column with reviews

    Return
    ------
    df: `pd.DataFrame`
        An updated DataFrame with the non-alphabetical characters removed
    """
    


    updated_reviews = []
    updated_df = df.copy()
    text_reviews = df[col_name].values
    # print(text_reviews)

    for text_reviews_idx in range(len(text_reviews)):
        text_review = text_reviews[text_reviews_idx]
        # print(text_review)

        # if type(text_review) != str:
        #     updated_reviews.append(text_review)
        # else:
        update_text_review = re.sub(r'[^a-zA-Z\s]', '', text_review)
        # print(text_review)
        updated_reviews.append(update_text_review)

    updated_df[col_name] = updated_reviews
    return updated_df

In [33]:
reviews_alph_chars_df = remove_non_alphabetical_characters(reviews_no_html_urls_rb_df, 'review_headline_lower_cased')

In [34]:
reviews_alph_chars_df

,star_rating,review_headline,review_body,positive_sentiment,negative_sentiment,review_headline_lower_cased,review_body_lower_cased
1075394,5.0,SammyStuff,Love this shredder. It is durable and can han...,1.0,NaN,sammystuff,love this shredder. it is durable and can han...
488457,5.0,Very good I like the black pen,Very good I like the black pen,1.0,NaN,very good i like the black pen,very good i like the black pen
633349,5.0,Very pleased with my purchase! The colors are ...,2014-10-04,1.0,NaN,very pleased with my purchase the colors are v...,2014-10-04
579734,4.0,Four Stars,Good small tool case for the &#34;glove compar...,1.0,NaN,four stars,good small tool case for the &#34;glove compar...
1060819,5.0,perfect,had problem finding this for my son at stores....,1.0,NaN,perfect,had problem finding this for my son at stores....
...,...,...,...,...,...,...,...
360704,0.0,I get the gron part namber i need you sand me ...,2015-02-26,NaN,0.0,i get the gron part namber i need you sand me ...,2015-02-26
822917,0.0,tremendous static not present on phone replace...,2014-04-17,NaN,0.0,tremendous static not present on phone replace...,2014-04-17
327341,0.0,Cute and a good size for holding a decent amou...,makeup brushes and such.,NaN,0.0,cute and a good size for holding a decent amou...,makeup brushes and such.
673490,1.0,Loving this product and the fact that I can us...,2014-09-09,NaN,0.0,loving this product and the fact that i can us...,2014-09-09


In [35]:
reviews_alph_chars_rb_df = remove_non_alphabetical_characters(reviews_alph_chars_df, 'review_body_lower_cased')

In [36]:
reviews_alph_chars_rb_df

,star_rating,review_headline,review_body,positive_sentiment,negative_sentiment,review_headline_lower_cased,review_body_lower_cased
1075394,5.0,SammyStuff,Love this shredder. It is durable and can han...,1.0,NaN,sammystuff,love this shredder it is durable and can hand...
488457,5.0,Very good I like the black pen,Very good I like the black pen,1.0,NaN,very good i like the black pen,very good i like the black pen
633349,5.0,Very pleased with my purchase! The colors are ...,2014-10-04,1.0,NaN,very pleased with my purchase the colors are v...,
579734,4.0,Four Stars,Good small tool case for the &#34;glove compar...,1.0,NaN,four stars,good small tool case for the glove compartment
1060819,5.0,perfect,had problem finding this for my son at stores....,1.0,NaN,perfect,had problem finding this for my son at storesw...
...,...,...,...,...,...,...,...
360704,0.0,I get the gron part namber i need you sand me ...,2015-02-26,NaN,0.0,i get the gron part namber i need you sand me ...,
822917,0.0,tremendous static not present on phone replace...,2014-04-17,NaN,0.0,tremendous static not present on phone replace...,
327341,0.0,Cute and a good size for holding a decent amou...,makeup brushes and such.,NaN,0.0,cute and a good size for holding a decent amou...,makeup brushes and such
673490,1.0,Loving this product and the fact that I can us...,2014-09-09,NaN,0.0,loving this product and the fact that i can us...,


## Remove extra spaces
- [ ] Verify with a specific entry

In [37]:
def remove_extra_spaces(df:pd.DataFrame, col_name: str):
    """Remove extra spaces from all reviews

    Parameters
    ----------
    df: `pd.DataFrame`
        The data
    
    col_name: `str`
        Column with reviews

    Return
    ------
    df: `pd.DataFrame`
        An updated DataFrame with the extra spaces removed
    """
    
    updated_reviews = []
    updated_df = df.copy()
    text_reviews = df[col_name].values
    # print(text_reviews)

    for text_reviews_idx in range(len(text_reviews)):
        text_review = text_reviews[text_reviews_idx]
        # print(text_review)

        # if type(text_review) != str:
        #     updated_reviews.append(text_review)
        # else:
        update_text_review = re.sub(r' +', ' ', text_review)
        # print(text_review)
        updated_reviews.append(update_text_review)

    updated_df[col_name] = updated_reviews
    return updated_df

In [38]:
reviews_no_extra_space_df = remove_extra_spaces(reviews_alph_chars_rb_df, 'review_headline_lower_cased')

In [39]:
# reviews_no_extra_space_df

In [46]:
reviews_no_extra_space_rb_df = remove_extra_spaces(reviews_no_extra_space_df, 'review_body_lower_cased')

In [47]:
reviews_no_extra_space_rb_df

,star_rating,review_headline,review_body,positive_sentiment,negative_sentiment,review_headline_lower_cased,review_body_lower_cased
1075394,5.0,SammyStuff,Love this shredder. It is durable and can han...,1.0,NaN,sammystuff,love this shredder it is durable and can handl...
488457,5.0,Very good I like the black pen,Very good I like the black pen,1.0,NaN,very good i like the black pen,very good i like the black pen
633349,5.0,Very pleased with my purchase! The colors are ...,2014-10-04,1.0,NaN,very pleased with my purchase the colors are v...,
579734,4.0,Four Stars,Good small tool case for the &#34;glove compar...,1.0,NaN,four stars,good small tool case for the glove compartment
1060819,5.0,perfect,had problem finding this for my son at stores....,1.0,NaN,perfect,had problem finding this for my son at storesw...
...,...,...,...,...,...,...,...
360704,0.0,I get the gron part namber i need you sand me ...,2015-02-26,NaN,0.0,i get the gron part namber i need you sand me ...,
822917,0.0,tremendous static not present on phone replace...,2014-04-17,NaN,0.0,tremendous static not present on phone replace...,
327341,0.0,Cute and a good size for holding a decent amou...,makeup brushes and such.,NaN,0.0,cute and a good size for holding a decent amou...,makeup brushes and such
673490,1.0,Loving this product and the fact that I can us...,2014-09-09,NaN,0.0,loving this product and the fact that i can us...,


## Remove Contractions
- [ ] Need to update; make my own

In [48]:
contraction_mapping = {
    "ain't": "am not",
    "aren't": "are not",
    "can't": "cannot",
    "don't": "do not"
}

In [49]:
def expand_contractions(input_text):
    # Function to replace contractions with their expanded forms
    def replace(match):
        print("match:", match)
        return contraction_mapping[match.group(0)]

    # Use regular expression to find contractions and replace them
    contraction_pattern = re.compile('({})'.format('|'.join(contraction_mapping.keys())), flags=re.IGNORECASE | re.DOTALL)
    expanded_text = contraction_pattern.sub(replace, input_text)

    return expanded_text

In [50]:
def remove_contractions(df:pd.DataFrame, col_name: str):
    """Remove contractions from all reviews

    Parameters
    ----------
    df: `pd.DataFrame`
        The data
    
    col_name: `str`
        Column with reviews

    Return
    ------
    df: `pd.DataFrame`
        An updated DataFrame with the extra spaces removed
    """
    
    updated_reviews = []
    updated_df = df.copy()
    text_reviews = df[col_name].values
    # print(text_reviews)

    for text_reviews_idx in range(len(text_reviews)):
        text_review = text_reviews[text_reviews_idx]
        # print(text_review)

        # if type(text_review) != str:
        #     updated_reviews.append(text_review)
        # else:
        without_contraction = expand_contractions(text_review)
        # print(text_review)
        updated_reviews.append(without_contraction)

    updated_df[col_name] = updated_reviews
    return updated_df

In [51]:
reviews_no_contractions_df = remove_contractions(reviews_alph_chars_rb_df, 'review_headline_lower_cased')

In [52]:
# reviews_no_contractions_df

In [53]:
reviews_no_contractions_rb_df = remove_contractions(reviews_no_contractions_df, 'review_body_lower_cased')

In [57]:
reviews_no_contractions_rb_df

,star_rating,review_headline,review_body,positive_sentiment,negative_sentiment,review_headline_lower_cased,review_body_lower_cased
1075394,5.0,SammyStuff,Love this shredder. It is durable and can han...,1.0,NaN,sammystuff,love this shredder it is durable and can hand...
488457,5.0,Very good I like the black pen,Very good I like the black pen,1.0,NaN,very good i like the black pen,very good i like the black pen
633349,5.0,Very pleased with my purchase! The colors are ...,2014-10-04,1.0,NaN,very pleased with my purchase the colors are v...,
579734,4.0,Four Stars,Good small tool case for the &#34;glove compar...,1.0,NaN,four stars,good small tool case for the glove compartment
1060819,5.0,perfect,had problem finding this for my son at stores....,1.0,NaN,perfect,had problem finding this for my son at storesw...
...,...,...,...,...,...,...,...
360704,0.0,I get the gron part namber i need you sand me ...,2015-02-26,NaN,0.0,i get the gron part namber i need you sand me ...,
822917,0.0,tremendous static not present on phone replace...,2014-04-17,NaN,0.0,tremendous static not present on phone replace...,
327341,0.0,Cute and a good size for holding a decent amou...,makeup brushes and such.,NaN,0.0,cute and a good size for holding a decent amou...,makeup brushes and such
673490,1.0,Loving this product and the fact that I can us...,2014-09-09,NaN,0.0,loving this product and the fact that i can us...,


# Pre-processing

## remove the stop words 

In [ ]:
from nltk.corpus import stopwords
 

In [ ]:
def remove_stop_words(df:pd.DataFrame, col_name: str):
    """Remove contractions from all reviews

    Parameters
    ----------
    df: `pd.DataFrame`
        The data
    
    col_name: `str`
        Column with reviews

    Return
    ------
    df: `pd.DataFrame`
        An updated DataFrame with the extra spaces removed
    """
    
    updated_reviews = []
    updated_df = df.copy()
    text_reviews = df[col_name].values
    # print(text_reviews)

    for text_reviews_idx in range(len(text_reviews)):
        text_review = text_reviews[text_reviews_idx]
        # print(text_review)

        # if type(text_review) != str:
        #     updated_reviews.append(text_review)
        # else:
        
        updated_reviews.append(without_contraction)

    updated_df[col_name] = updated_reviews
    return updated_df

## perform lemmatization  

In [ ]:
from nltk.stem import WordNetLemmatizer


# TF-IDF Feature Extraction

# Perceptron

# SVM

# Logistic Regression

# Naive Bayes